# Cvičenie 4: Policy Iteration

Na minulom cvičení ste videli, ako sa dajú vyriešiť Markovovské rozhodovacie procesy pomocou Bellmanových rovníc. Od tohto týždňa sa pozrieme na rôzne metódy učenia posilňovaním, pričom ich budeme aplikovať na rovnaký gridworld problém pre porovnávanie. Prvá metóda bude práve Policy Iteration.

## Policy Iteration

Policy Iteration pozostáva z iteratívnej aplikácie dvoch krokov, a to vyhodnocovanie (*policy evaluation*) a zlepšovanie (*policy improvement*). Pseudokód algoritmu nájdete na obrázku nižšie.

<img src="lab04/policy_iteration.jpg" width="600">
<p style="text-align: center;">Zdroj: Sutton-Barto: Reinforcement Learning, 2nd ed., 2018</p>

Prediskutujte význam jednotlivých operácií a krokov a následne aplikujte algoritmus na ukážkový príklad gridworldu. Ako ukážku, nižšie nájdete vypracované kroky pre jeden stav.

<img src="lab04/gridworld_mdp.jpg" width="600">

Príklad predstavuje svet *3x3* s cieľovou pozíciou v pravom hornom rohu, a s jednou pascou v strede sveta. K dispozícii sú štyri akcie: posun na sever, východ, juh a západ. Ak sa hráč dostane do cieľa, obdrží odmenu 10, ak spadne do pasce, tak -10. V oboch prípadoch sa hra ukončí. Pre ostatné kroky dostane agent odmenu -1. Pre stav označený $s_{32}$ môže byť odmena 5 s pravdepodobnosťou *0.1*.

Väčšina sveta je úplne deterministická. Na pozícii $s_{12}$ fúka silný vietor, ktorý môže agenta posunúť na juh aj keď sa pohybuje iným smerom. Pravdepodobnosť pohybu vo vybranom smere je v týchto prípadoch $0.6$, pravdepodobnosť posunutia na juh je $0.4$. Ak agent vyberie pohyb na juh, určite sa tam dostane.

Discount factor $\gamma = 0.8$ a $\theta = 0.1$. Začneme s politikou $\pi(s) = N \ \ \forall \ \ s \in \mathcal{S}$, teda na každej pozícii sa vyberie pohyb na sever.

### 1. Policy Evaluation

Pre stav $s_{11}$ (prvý riadok, prvý stĺpec) vieme vypočítať hodnotu stavu tak, že berieme do úvahy všetky možné prechody pri výbere akcie $N$. Na základe pravdepodobností prechodov vieme vypočítať hodnotu tohto stavu:

$V(s_{11}) = p \left ( s_{11}, r_{s} | s_{11}, \pi(s_{11}) \right ) \cdot (r_{s} + \gamma \cdot V(s_{11})) $

$V(s_{11}) = 1.0 \cdot (-1 + 0.8 \cdot 0) $

$V(s_{11}) = -1$

Zmena odhadu pre $V(s_{11})$ je $\Delta V(s_{11}) = |0 - (-1)| = 1$, čo je väčšie ako $\theta$, takže už teraz je zrejmé, že vyhodnocovanie si budeme musieť zopakovať.

V našom svete pre nedeterministickú pozíciu $s_{12}$ existujú dva možné prechody: $s_{12}$ a $s_{22}$. Na základe pravdepodobností prechodov vieme vypočítať hodnotu tohto stavu:

$V(s_{12}) = p \left ( s_{12}, r_{s} | s_{12}, \pi(s_{12}) \right ) \cdot (r_{s} + \gamma \cdot V(s_{12})) + p \left ( s_{22}, r_{t} | s_{12}, \pi(s_{12}) \right ) \cdot (r_{t} + \gamma \cdot V(s_{22})) $

$V(s_{11}) = 0.6 \cdot (-1 + 0.8 \cdot 0) + 0.4 \cdot (-10 + 0.8 \cdot 0) $

$V(s_{11}) = -4.6$

Zmena odhadu pre $V(s_{12})$ je $\Delta V(s_{12}) = |0 - (-4.6)| = 4.6$, takže hodnotu najväčšej zmeny aktualizujeme.

**Úloha:** Vypočítajte hodnotu ďalších stavov (okrem $s_{13}$ a $s_{22}$). Použite pritom inplace prístup.

**Úloha:** Vypočítajte hodnotu ďalších stavov na základe nasledujúcich hodnôt, ktoré by ste dostali po niekoľkých iteráciách:

$V(s_{11}) = -2.952$

$V(s_{12}) = -8.377$

$V(s_{13}) = 0.0$

$V(s_{21}) = -3.362$

$V(s_{22}) = 0.0$

$V(s_{23}) = 10.0$

$V(s_{31}) = -3.689$

$V(s_{32}) = -10.0$

$V(s_{33}) = 7.0$

### 2. Policy Improvement

Začiatočnú politiku $\pi_0$ definujeme ľubovoľne, v našom prípade to bola politika posunu na sever v každom stave. Ak už máme vypočítane hodnoty stavov, tieto hodnoty vieme použiť pri zlepšovaní politiky. Pre každú akciu vypočítame očakávanú dlhodobú odmenu podľa vzorca:

$q(s, a) = \sum_{s', r} p(s', r|s, a) \cdot (r + \gamma \cdot V(s'))$

Vyhodnocovanie politiky končilo nasledovne:

$V(s_{11}) = -3.362$

$V(s_{12}) = -8.621$

$V(s_{13}) = 0.0$

$V(s_{21}) = -3.689$

$V(s_{22}) = 0.0$

$V(s_{23}) = 10.0$

$V(s_{31}) = -3.951$

$V(s_{32}) = -10.0$

$V(s_{33}) = 7.0$

Teda pre stav $s_{12}$:

$q(s_{12}, N) = p(s_{12}, r_{s} | s_{12}, N) \cdot (r_{s} + \gamma \cdot V(s_{12})) + p(s_{22}, r_{t} | s_{12}, N) \cdot  (r_{t} + \gamma \cdot V(s_{22})) = 0.6 \cdot (-1 + 0.8 \cdot -8.621) + 0.4 \cdot (-10 + 0.8 \cdot 0) = -8.738 $

$q(s_{12}, E) = p(s_{13}, r_{g} | s_{12}, E) \cdot (r_{g} + \gamma \cdot V(s_{13})) + p(s_{22}, r_{t} | s_{12}, E) \cdot  (r_{t} + \gamma \cdot V(s_{22})) = 0.6 \cdot (10 + 0.8 \cdot 0) + 0.4 \cdot (-10 + 0.8 \cdot 0) = 2.0 $

$q(s_{12}, S) = p(s_{22}, r_{t} | s_{12}, S) \cdot  (r_{t} + \gamma \cdot V(s_{22})) = 1.0 \cdot (-10 + 0.8 \cdot 0) = -10 $

$q(s_{12}, W) = p(s_{11}, r_{s} | s_{12}, W) \cdot (r_{s} + \gamma \cdot V(s_{11})) + p(s_{22}, r_{t} | s_{12}, W) \cdot  (r_{t} + \gamma \cdot V(s_{22})) = 0.6 \cdot (-1 + 0.8 \cdot (-3.362)) + 0.4 \cdot (-10 + 0.8 \cdot 0) = -6.214 $

pre jednotlivé akcie. Následne aplikujeme funkciu $argmax$, ktorá nám vráti akciu, pri ktorej je hodnota maximálna:

$\pi(s_{12}) = argmax_a(q(s_{12}, N), q(s_{12}, E), q(s_{12}, S), q(s_{12}, W)) = E$

**Úloha:** Vypočítajte všetky očakávané odmeny pre dvojice stav-akcia (okrem stavov $s_{13}$ a $s_{22}$) a na základe hodnôt aktualizujte politiku $\pi$.

## Implementácia

Vaše výpočty overte na [ukážkovej implementácii metódy Policy Iteration](lab04/policy_iteration.py) a spozorujte, ako sa metóda dopracuje k optimálnej politike.

**Domáca úloha:** Implementáciu opravte tak, aby namiesto inplace stratégie používala sweep stratégiu.